In [1]:
from tqdm import tqdm
from model.block import Block, BlockWithEmbedding
from layers.loader.book_loader import BookLoader
from layers.loader.filtering_book_loader import FilteringBookLoader
from layers.segmentation.block_segmenter import BlockSegmenter
from layers.embedding.sentence_transformer_embedding import SentenceTransformerEmbedding
from layers.persistance import duckdb_storage
from model.book import Book
from layers.segmentation.text_segmenter import TextSegmenter

In [2]:
loader: BookLoader = FilteringBookLoader(filters=[])
books: list[Book] = list(loader.read("data/"))

In [3]:
segmenter: TextSegmenter = BlockSegmenter(block_max_length=2000)

In [4]:
with SentenceTransformerEmbedding(
        model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
        device="cuda"
) as embedding_model:
    with duckdb_storage.DuckDBStorage("data.db") as storage:
        for book in tqdm(books):
            blocks: list[Block] = segmenter.process(book.text)
            blocks_with_embeddings: list[BlockWithEmbedding] = embedding_model.transform(blocks)
            storage.add_book(book, blocks_with_embeddings)

/home/yshmgrt/Programming/accurate-text-generation/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/yshmgrt/Programming/accurate-text-generation/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
  2%|▏         | 74/3575 [09:14<7:17:00,  7.49s/it] 


FatalException: FATAL Error: Failed to create checkpoint because of error: Could not remove file "/home/yshmgrt/Programming/accurate-text-generation/data.duckdb.wal": No such file or directory